## **_Enterprise Data Science and Analytics - Enterprise Data Science Bootcamp_**

### **HR Attrition Project - EDSB25_26**

- Ana Rita Martins 20240821
- Joana Coelho 20240801
- Pedro Fernandes 20240823
- Ricardo Silva 20240824

Data Science and Analytics are reshaping how organizations solve problems across diverse industries. Through systematic data analysis and predictive modeling, evidence-based solutions can be developed, enabling more reliable decision-making and greater efficiency.

In Human Resources, predictive analytics supports critical functions such as employee retention, workforce planning, and automated CV screening.

This project focuses on developing predictive models to assess the likelihood of employee resignation. By analyzing factors ranging from demographics to job satisfaction, the models aim to provide interpretable insights that highlight key drivers of attrition. These insights will help HR leaders take proactive steps to reduce turnover and retain talent.

## 1. Importing Packages

In [ ]:
import pandas as pd
from summarytools import dfSummary
import matplotlib.pyplot as plt
import seaborn as sns
#from pandas.io.formats.style import Styler

## 2. Importing Data and Initial Exploration

In [ ]:
data = pd.read_csv('../data/raw/HR_Attrition_Dataset.csv')
print(data.head())

In [ ]:
data.info()

In [ ]:
pd.set_option('display.max_columns', None) 
data.describe() 


In [ ]:
data.describe(include='object')

From this initial inspection what immediately stands out is that we have 3 constant features: "EmployeeCount", "StandardHours", and "Over18". We can remove those straight away. Additionally, the employee number (ID) feature, does not seem to contain any relevant info, and  we'll drop it too.

In [ ]:
data.drop(columns=['EmployeeCount','Over18','StandardHours','EmployeeNumber'],inplace=True)

In [ ]:
cat_cols = data.select_dtypes(include=["object"]).columns

for col in cat_cols: 
    print(f"Value counts for column '{col}':")
    print(data[col].value_counts())
    print("\n") 

In [ ]:
dfSummary(data)

From the summary above, we verified that the data set does't contain duplicates, and we also gathered information about the data's distribution and main statistics.

What we can note is that, beasides our target, we have a couple of other binary features. Let's encode those.

In [ ]:
data['Attrition'] = data['Attrition'].map({'Yes': 1, 'No': 0})
data['Gender'] = data['Gender'].map({'Male': 1, 'Female': 0})
data['OverTime'] = data['OverTime'].map({'Yes': 1, 'No': 0})

data.head()

Let's now have a look at how the distribution of the target variable.

In [ ]:
ax = sns.countplot(x=data['Attrition'], hue=data['Attrition'], legend=False)
for container in ax.containers:
    ax.bar_label(container)

plt.title('Distribution of the Target Variable (Attrition)')
plt.show()

We can observe that our target cariable is quite imbalanced. This will require extra attention in later steps, namely when splitting the dataset into train, validation and test sets, as well as during the modelling stage.

In [ ]:
data.shape

In [ ]:
data.head(3)

# **3. Exploratory Data Analysis**

We'll start by plotting histograms to visually assess the distribution of the numeric features; this will allows us to spot any relevant patterns or trends in the data.

In [ ]:
data.hist(figsize=(20, 15))
plt.show()

The histograms reveal some important patterns in the dataset. 
- Once again we can observe that the **target variable** is highly skewed toward staying in the company.
- Concerning demographics, **age** follows an approximately bell-shaped distribution, centered around 30-40; **Gender** is skewed with more males than females.
- Features that are related to **work characteristics** (YearsAtCompany, TotalWorkingYears, YearsInCurrentRole, Overtime) are right-skewed, indicating many relatively new employees and fewer with long careers; working overtime is not common.
- **Income**: Salaries and rates are right-skewed, with few very high earners.
- **Satisfaction-related** variables are discrete and somewhat skewed toward higher ratings, while PerformanceRating shows very little variation (nearly all at level 3), suggesting limited predictive value.

Overall, the data displays strong imbalance and skewness patterns that will require careful consideration during modeling, suggesting it could benefit from stratified splits, and algorithms robust to class imbalance. 

In [ ]:
# Selecting  numerical columns (binaries excluded)
binary_cols = ['Attrition', 'Gender', 'OverTime']
num_cols = [col for col in data.select_dtypes(include=['int64', 'float64']).columns if col not in binary_cols]

# Boxplots for each numerical feature
n_cols = 5
n_rows = -(-len(num_cols) // n_cols)  

plt.figure(figsize=(20, 4*n_rows))

for i, col in enumerate(num_cols, 1):
    plt.subplot(n_rows, n_cols, i)
    sns.boxplot(y=data[col])
    plt.title(col)

plt.tight_layout()
plt.show()

The boxplots highlight the extent of skewness and make the outliers stand out clearly, which complements the histogram analysis above.
- Outliers are especially relevant in income and emplyment duration related-variables, which may need special handling. We'll decide how to handle them further down.
- For demographic/job characteristics (Age, DistanceFromHome, JobLevel, Education) featured the distributions are fairly compact with few outliers, aligning with the unimodal/bell-like shapes seen in histograms.
- Ordinal satisfaction and variables show limited spread, consistent with their discrete scale, with some level of skew toward higher values. Their limited range may reduce their explanatory power.
- PerformanceRating shows very little variation (nearly all values at level 3) confirming its limited usefulness as a predictive feature.

In [ ]:
# Selecionar apenas colunas do tipo object (qualitativas)
categorical_cols = data.select_dtypes(include='object').columns.tolist()

# Remover a variável Attrition da lista
if 'Attrition' in categorical_cols:
    categorical_cols.remove('Attrition')

# Criar gráficos para cada variável qualitativa
for col in categorical_cols:
    plt.figure(figsize=(10, 6))
    
    # Criar tabela de contingência
    contingency = data.groupby([col, 'Attrition']).size().unstack(fill_value=0)
    
    ax = sns.countplot(x=data[col], hue=data['Attrition'])
    
    for container in ax.containers:
        labels = []
        for i, bar in enumerate(container):
            height = bar.get_height()
            if height > 0:
                # Obter a categoria pela posição da barra
                categories = data[col].unique()
                category = categories[i] if i < len(categories) else categories[0]
                category_total = len(data[data[col] == category])
                percentage = (height / category_total) * 100
                labels.append(f'{percentage:.1f}%\n(n={int(height)})')
            else:
                labels.append('')
        ax.bar_label(container, labels=labels)
    
    plt.title(f'Distribution of {col} by Attrition')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

By analyzing these bar charts, we can identify notable differences in the distribution of Attrition across our categorical variables. Key findings include:

- **Business Travel**: Higher travel frequency might correlate with increased attrition rates, considering the differences between their percentages. This may suggest that frequent business travel may negatively impact employee retention.

- **Education Field and Job Role**: While some correlation between education field and job role is probable (Human Resources, for instance), the Job Role variable reveals a striking pattern: Sales Representatives show nearly 40% attrition rate, significantly higher than other positions.

- **Marital Status**: Single employees demonstrate substantially higher attrition rates compared to their married counterparts. The proportion of single employees leaving the company is more than double that of married employees (relative to their respective group sizes), indicating that marital status may be a relevant predictor of attrition.

These preliminary observations suggest that work-life balance factors (travel, marital status) and specific job roles warrant further investigation in our analysis.

We will also analyze the numerical variables in our dataset. However, before proceeding, it would be interesting to explore a potential relationship between MonthlyIncome and MonthlyRate. While MonthlyIncome represents the actual salary an employee receives each month, MonthlyRate might reflect the hourly rate or the standardized value the company attributes to that employee on a monthly basis. 

To investigate this relationship, we propose creating a new variable - Income_Rate_Ratio - which will capture the ratio between these two metrics and may provide insights into compensation structures and their potential impact on attrition.

In [ ]:
data['Income_Rate_Ratio'] = data['MonthlyIncome'] / data['MonthlyRate']

In [ ]:
# Dicionário para guardar as percentagens
percentage_dict = {}

numeric_cols = data.select_dtypes(include=['int64', 'float64']).columns.tolist()

for col in numeric_cols:
    plt.figure(figsize=(8, 5))
    ax = sns.barplot(x=data['Attrition'], y=data[col], errorbar='sd')

    # Remover a variável Attrition da lista
    if 'Attrition' in numeric_cols:
        numeric_cols.remove('Attrition')

    # Calcular médias
    means = data.groupby('Attrition')[col].mean()
    
    mean_no = means.get('No', means.iloc[0])
    mean_yes = means.get('Yes', means.iloc[1])
    
    # Calcular diferença e percentagem
    difference = mean_yes - mean_no
    percentage = ((mean_yes / mean_no) - 1) * 100 if mean_no != 0 else 0
    
    # Guardar percentagem
    percentage_dict[col] = percentage
    
    # Adicionar valores das médias
    for container in ax.containers:
        ax.bar_label(container, fmt='%.2f')
    
    # Adicionar texto com a relação entre médias
    plt.text(0.5, 0.95, f'Difference: {difference:.2f} | Change: {percentage:.1f}%', 
             transform=ax.transAxes, ha='center', va='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.title(f'Mean {col} by Attrition')
    plt.tight_layout()
    plt.show()

# Mostrar top 10 variáveis com maior percentagem (em valor absoluto)
top_10 = pd.Series(percentage_dict).abs().sort_values(ascending=False).head(10)
print("\nTop 10 variáveis com maior variação percentual:")
print(top_10)

By analyzing the variables with the largest percentage differences between attrition outcomes, several key patterns emerge:

- **OverTime** demonstrates by far the highest percentage variation (128.6%), more than three times greater than any other variable. This suggests that overtime work is the single most critical factor associated with employee attrition and warrants immediate attention in retention strategies.

- **Stock Options** show a substantial 37.6% variation, with employees possessing stock options exhibiting significantly lower attrition rates. This suggests that equity compensation may serve as an effective retention strategy, by strengthening organizational commitment and long-term alignment with the company's success.

- **Tenure-related variables** (Years in Current Role: 35.3%, Years with Current Manager: 34.7%, Years at Company: 30.4%, Total Working Years: 30.5%) consistently rank among the top differentiators. Employees with higher tenure across these dimensions show lower attrition rates, indicating that role stability, manager continuity, and organizational tenure contribute positively to retention.

- The newly created **Income_Rate_Ratio** (30.7%) exhibits greater percentage variation than either MonthlyIncome (29.9%) or MonthlyRate alone. This suggests that the relationship between these compensation metrics may reveal misalignment between employee remuneration and their perceived organizational value, potentially contributing to attrition decisions.

However, these observations are preliminary and reflect observable relationships. Statistical testing and multivariate analysis will be necessary to establish the strength and significance of these relationships while controlling for potential confounding variables and interactions.